In [ ]:
# referencias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc 
import math
from pandas_profiling import ProfileReport
from sklearn.cluster import KMeans
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Por questões de segurança a conexão disponibilizada foi descaracterizada e apontada posteriormente para csv do dataset relacionado
''' 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=XXXXXXXXXX;'
                      'Database=YYYYYYYY;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
dataset = pd.read_sql_query('ZZZZZZZZZZZZ',conn)
'''
# atribuindo dataset
dataset = pd.read_csv("dataset_peneg.csv")

In [ ]:
df_transact = dataset[['CANAL_ATENDIMENTO','TEMA_ATENDIMENTO','ABORDAGEM_ATENDIMENTO','CATEGORIA_ATENDIMENTO','INSTRUMENTO_ATENDIMENTO','MEIO_ATENDIMENTO']]

In [ ]:
df_transact.describe()

In [ ]:
sns.pairplot(df_transact)

###### Executando K-Means inicialmente com agrupamentos aleatórios (1 para cada par de variáveis) para obter o número ideal posteriormente

In [ ]:
# Função para calculo do somatório da variância dos dados em relação ao número de clusters iniciais
def calculate_wcss(data):
    wcss = []
    # 3 iterações por um cluster a cada par dos 6 atributos possíveis
    for n in range(1, 4):
        kmeans = KMeans(n_clusters=n,init='k-means++')
        kmeans.fit(X=data)
        print (n,kmeans.inertia_)
        wcss.append(kmeans.inertia_)
    return wcss #Within Cluster Sum of Squares (soma dos quadrados do cluster)

In [ ]:
# Chamando função para somatório da variância dos dados em relação ao número de clusters (soma dos quadrados de cada clusters)
sum_of_squares = calculate_wcss(df_transact)

In [ ]:
# Função para verificar até que ponto com o aumento do nº de clusters não existe ganho para se chegar ao número ideal
# Baseada na formula do cálculo da distância entre um ponto e uma reta
def optimal_number_of_clusters(wcss):
    x1, y1 = 1, wcss[0]
    x2, y2 = 6, wcss[len(wcss)-1]
    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = math.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    return distances.index(max(distances))

In [ ]:
# Calculando a quantidade ótima de clusters (ideal) e atribuindo a variavel para utilização futura
n = optimal_number_of_clusters(sum_of_squares)

###### Verificando número ideal de clusters com inferência gráfica do Método Elbow (Cotovelo)

In [ ]:
# Verificando a quantidade ideal de cluster calculadas com base e plot em gráfico de cotovelo
print('     Nº ideal de clusters calculado: ', n)
plt.plot(range(1, 4), sum_of_squares)
plt.title('Metodo Elbow')
plt.xlabel('Nº Clusters')
plt.ylabel('WSS') #Within Cluster Sum of Squares (soma dos quadrados do cluster)
plt.show()

###### Executando K-Means novamente, porém, desta vez com o número ideal de clusters

In [ ]:
# Iniciando KMeans com numero ideal de clusters (n calculado)
kmeans = KMeans(n_clusters = n, init = 'k-means++', random_state = 0)
# Aplicando o algoritmo
kmeans.fit(df_transact)

###### Analisando clusters atribuídos

In [ ]:
# Atribuindo clusters ao dataset/dataframe 
### labels são o código de cada cluster criado pelo K-Means
df_transact['CLUSTER'] = kmeans.labels_

In [ ]:
df_transact.groupby("CLUSTER").aggregate("mean").plot.bar(figsize=(20,8))
plt.title("Distribuição por Clusters")

In [ ]:
sns.pairplot(df_transact,hue='CLUSTER', palette="tab10", plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'})

In [ ]:
df_transact.loc[df_transact['CLUSTER']==0].describe()

In [ ]:
df_transact.loc[df_transact['CLUSTER']==1].describe()

In [ ]:
sns.color_palette("tab10")
sns.pairplot(
    df_transact,
    vars = ['MEIO_ATENDIMENTO', 'CANAL_ATENDIMENTO'],
    hue='CLUSTER',
    palette="tab10",
    plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'}
)

In [ ]:
# Visualizando centroides por clustering de idade
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
predict_kmeans = kmeans.predict(df_transact.drop('CLUSTER',axis=1))
#plt.scatter(dataset[:, 0], dataset[:, 12], c=predict_kmeans, s=50, cmap='CLUSTER')
plt.scatter(df_transact['INSTRUMENTO_ATENDIMENTO'], df_transact['INSTRUMENTO_ATENDIMENTO'], c=predict_kmeans, s=50, cmap='plasma')
plt.scatter(centroids[:, 4], centroids[:, 4], c='red', s=200, alpha=0.5);

In [ ]:
df_transact[['CANAL_ATENDIMENTO','TEMA_ATENDIMENTO',
             'ABORDAGEM_ATENDIMENTO','CATEGORIA_ATENDIMENTO',
             'INSTRUMENTO_ATENDIMENTO','MEIO_ATENDIMENTO']].corr().style.background_gradient(cmap='coolwarm', axis=None)